# Filtracja Non-Local Means

## Definicja

Kolejny "poziom wtajemniczenia" w zagadnienie filtracji obrazów to metoda Non-Local Means (NLM).
Została ona zaproponowana w pracy *A non-local algorithm for image denoising* autorstwa Antoni Buades, Bartomeu Coll, i Jean Michel Morel na konferencji CVPR w 2005 roku.

Filtr NLM dany jest zależnością:

\begin{equation}
\hat{I}(\mathbf{x}) = \sum_{\mathbf{p} \in V(\mathbf{x})} w(\mathbf{p},\mathbf{x})I(\mathbf{p})
\end{equation}

gdzie:
- $I$ - obraz wejściowy,
- $\hat{I}$ - obraz wyjściowy (przefiltrowany),
- $\mathbf{x}$ - współrzędne piksela obrazu,
- $V(\mathbf{x})$ - obszar poszukiwań piksela, dla którego przeprowadzana jest filtracja,
- $w$ - waga punktu $\mathbf{p}$ z obszaru poszukiwań.

Wróćmy na chwilę do filtracji bilateralnej. Tam waga danego piksela z kontekstu zależała od dwóch czynników - odległości przestrzennej pomiędzy pikselami oraz różnicy w jasności/kolorze pomiędzy pikselami (tzw. przeciwdziedzina).
Filtr NLM stanowi uogólnienie tej metody - do obliczania wag nie wykorzystuje się już pojedynczych pikseli ($\mathbf{p}$ i $\mathbf{x}$), a lokalne konteksty ($N(\mathbf{p})$ i $N(\mathbf{x})$).

Waga $w$ dana jest następującą zależnością:

\begin{equation}
w(\mathbf{p},\mathbf{x}) = \frac{1}{Z(\mathbf{x})}\exp(-\frac{|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||^2_{2}}{\alpha \sigma^2})
\end{equation}

gdzie:
- \begin{equation}
Z(\mathbf{x}) = \sum_{\mathbf{p} \in  V(\mathbf{x})} \exp(-\frac{|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||^2_{2}}{\alpha \sigma^2})
\end{equation},
- $|| \cdot ||$ - jest normą $L_2$ odległości pomiędzy dwoma kontekstami,
- $v$ oznacza mnożenie punktowe kontekstu $N$ przez dwuwymiarową maskę Gaussa o odpowiadających kontekstowi wymiarach,
- $\alpha$ > 0 - parametr sterujący filtracją,
- $\sigma$ - parametr szumu stacjonarnego występującego na obrazie (w przypadku szumu niestacjonarnego, parametr $\sigma$ musi zostać dopasowany lokalnie tj. $\sigma = \sigma(\mathbf{x})$).

## Analiza działania

Zastanówmy sie teraz jak działa filtra NLM. Najprościej to zrozumieć na rysunku.

![Ilustracja NLM](https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/nlm.png)

1. Dla rozważanego piksela $\mathbf{x}$ definiujemy obszar poszukiwań $V(\mathbf{x})$. Uwaga - obszar poszukiwań ($V$) jest jednostką większą niż otocznie/kontekst ($N$).

2. Następnie, dla każdego z pikseli $\mathbf{p} \in  V(\mathbf{x})$ oraz samego $\mathbf{x}$ definiujemy otocznie/kontekst odpowiednio $N(\mathbf{p})$ i $N(\mathbf{x})$.

3. Wracamy do równania definiującego wagę  $w(\mathbf{p},\mathbf{x})$, a konkretnie do wyrażenia $|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||$. Przeanalizujmy co ono oznacza. Mamy dwa otoczenia: $N(\mathbf{p})$ i $N(\mathbf{x})$. Każde z nich mnożymy przez odpowiadającą maskę Gaussa - funkcja $v$. Otrzymujemy dwie macierze, które odejmujemy od siebie punktowo. Następnie obliczamy kwadrat z normy ($L_2$ definiujemy jako $||X||_2 = \sqrt{\sum_k|X_k|^2}$. Otrzymujemy zatem jedną liczbę, która opisuje nam podobieństwo otoczeń pikseli $\mathbf{x}$ i $\mathbf{p}$. Mała wartość oznacza otoczenia zbliżone, duża - różniące się. Ponieważ, z dokładnością do stałych, liczba ta stanowi wykładnik funkcji $e^{-x}$, to ostatecznie waga jest zbliżona do 1 dla otoczeń podobnych, a szybko maleje wraz z malejącym podobieństwem kontekstów.

4. Podsumowując. Jak wynika z powyższej analizy filtr NLM to taki filtr bilateralny, w którym zamiast pojedynczych pikseli porównuje się ich lokalne otoczenia. Wpływa to pozytywnie na jakość filtracji, niestety kosztem złożoności obliczeniowej.

## Implementacja

W ramach zadania należy zaimplementować filtr NLM, ocenić jego działanie w porównaniu do filtra Gaussa i bilateralnego oraz dokonać pomiaru czasu obliczeń (dla trzech wymienionych metod).

Jak już się zrozumie jak działa NLM, jego implementacja jest dość prosta.
Wartość parametru $\alpha$ należy dobrać eksperymentalnie.
Nie należy także "przesadzić" z rozmiarem obszaru poszukiwań (np. 11x11) oraz kontekstu (5x5 lub 3x3).

Wskazówki do implementacji:
- algorytm sprowadza się do dwóch podwójnych pętli for: zewnętrzne po pikselach, wewnętrzne po kolejnych obszarach przeszukań,
- przed realizacją trzeba przemyśleć problem pikseli brzegowych - de facto problemów jest kilka. Po pierwsze nie dla każdego piksela można wyznaczyć pełny obszar przeszukań (tu propozycja, aby filtrację przeprowadzać tylko dla pikseli z pełnym obszarem). Po drugie, ponieważ rozpatrujemy konteksty, to nawet dla piksela o "pełnym" obszarze przeszukań, będą istnieć piksele, dla których nie pełnych kontekstów (sugestia - powiększyć obszar przeszukać, tak aby zawierał konteksty). Ostatni problem jest bardziej techniczny/implementacyjny. Jeśli w kolejnych iteracjach "jawnie" wytniemy fragment o rozmiarach obszaru przeszukiwań, to znowu pojawi się problem brzegowy - tu można albo wyciąć nieco większy obszar, albo cały czas "pracować" na obrazie oryginalnym ("żonglerka indeksami").
- warto sprawdzać indeksy i rozmiary "wycinanych" kontekstów,
- wagi wyliczamy w trzech krokach:
    - obliczenia dla $N(\mathbf{x})$ + inicjalizacja macierzy na wagi,
    - podwójna pętla, w której przeprowadzamy obliczenia dla kolejnych $N(\mathbf{p})$ oraz wyliczamy wagi,
    - normalizacja macierzy wag oraz końcowa filtracja obszaru w wykorzystaniem wag.
- uwaga, obliczenia trochę trwają, nawet dla obrazka 256x256 i względnie niewielkich obszaru przeszukań i kontesktu.

Efekt końcowy:
- porównanie wyników metod: filtr Gaussa, filtr bilateralny oraz filtr NLM (2-3 zdania komentarza),
- porównanie czasu działania powyższych metod (1 zdanie komentarza).


In [ ]:
def fgaussian(size, sigma):
    m = n = size
    h, k = m//2, n//2
    x, y = np.mgrid[-h:h+1, -k:k+1]
    g = np.exp(-(x**2 + y**2)/(2*sigma**2))
    return g /g.sum()

##########################################################################################################

def newPixelValue(window,mask):
    X, Y = window.shape
    output = np.zeros(window.shape)
    for i in range(X):
        for j in range(Y):
            output[i,j] = window[i,j]*mask[i,j]
    return np.sum(output)/np.sum(mask)

def conv(img,windowSize,sigma):
    
    mask = fgaussian(windowSize,sigma)
   # mesh(mask, windowSize)
    
    IConv = img.copy()
    H,W = img.shape
    a = windowSize//2

    for i in range(a,H-a):
        for j in range(a,W-a):
            content = img[i-a : i+a+1, j-a : j+a+1]
            
            IConv[i,j]= newPixelValue(content,mask)
            
    return IConv

##########################################################################################################

def gaussian(x, sigma):
    return np.exp(((-1)*x)/(2*(sigma)**2));

def distance(x,y,i,j):
    return ((x-i)**2+(y-j)**2)**0.5

def newPixelValueBilateral(window,mask,sigmar):
    X, Y = window.shape
    output = np.zeros(window.shape)
    pixel = [X//2,Y//2]
#     print(pixel)
    
    for i in range(X):
        for j in range(Y):
            dst = distance(pixel[0],pixel[1],i,j)
           
            gamma = gaussian(dst,sigmar)
            output[i,j] = gamma* window[i,j]*mask[i,j]
            
    return np.sum(output)/np.sum(mask)
     
def bilateralFilter(img,windowSize,sigmas,sigmar):
    mask = fgaussian(windowSize,sigmas)
    IBilateral = img.copy()
    H,W = img.shape
    a = windowSize//2
    for i in range(a,H-a):
        for j in range(a,W-a):
            content = img[i-a : i+a+1, j-a : j+a+1]
                         
            IBilateral[i,j]= newPixelValueBilateral(content,mask,sigmar)
            
    return IBilateral

In [ ]:
def NonLocalMeansFilter(img, windowSize, contextSize, alfa, sigma):
    Inlm = img.copy()
    H,W = img.shape
    mask = fgaussian(contextSize, 5)
    
    border = windowSize//2 + contextSize//2
    a = windowSize//2
    b = contextSize//2 
    for i in range(border, H - border ):
        for j in range(border, W - border):
            content = img[i-a : i+a+1, j-a : j+a+1]
            
            Hc,Wc  = content.shape
            C = img[i - b : i + b + 1  , j - b : j + b+ 1]
            
            matrix = np.zeros(content.shape)
            
            for i_w in range(Hc):
                for j_w in range(Wc):

                    cnt= img[i - a + i_w -b : i - a +i_w + b +1, j - a + j_w - b : j - a + j_w + b + 1]
                    
                    diff = np.dot(C, mask) - np.dot(cnt, mask)  
                    diffnormalized = (np.sum(diff**2))**0.5        
                    normalized = diffnormalized**2
                    matrix[i_w,j_w] = gaussian(normalized,sigma)

            zx = np.sum(matrix)
            endmatrix = np.zeros(content.shape)
        
            for i_w in range(Hc):
                for j_w in range(Wc):
                    endmatrix[i_w, j_w] = (1/zx) * matrix[i_w,j_w] * img[i- a + i_w, j - a+ j_w ]
                
        
            value = np.sum(endmatrix )
            
            Inlm[i,j]= value
    return Inlm


In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from scipy.io import loadmat
import math
import os
from timeit import default_timer as timer

if not os.path.exists("MR_data.mat") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/MR_data.mat --no-check-certificate

plt.gray()
mat = loadmat('MR_data.mat')
Input = mat['I_noisy4']


start = timer()
convImage=conv(Input, 3,5)
end = timer()
convTime=(end - start)

start = timer()
bilateralImage=bilateralFilter(Input, 3, 5, 3)
end = timer()
bilateralTime=(end - start)

start = timer()
nlmImage=NonLocalMeansFilter(Input, 11, 3,15,3)  
end = timer()    
nlmTime=(end - start)


f, ([ax1,ax2],[ax3,ax4]) = plt.subplots(2,2, figsize=(20,20))
ax1.imshow(Input)
ax2.imshow(convImage)
ax3.imshow(bilateralImage)
ax4.imshow(nlmImage)